# Detect and Track Objects in Lyft Dataset and create Carmaker Files
This Notebook is for creating the required files and running the detector training and generating the simulation files
### It is necessary to run this Notebook in the lyft_env environment
First install the conda environment lyft_env using the lyft-environment.yaml file.
Activate the environment in Jupyter using: Kernel->Change Kernel->conda_env:lyft_env

In [49]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of second.data.nuscenes_dataset failed: Traceback (most recent call last):
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/itiv/Desktop/repo/scenarios_in_CarMaker/BA_Daniel/Lyft-Detector/second.pytorch/s

## Lyft-Detector Training
The Detector is trained using the train dataset. Model changes can be made in the config file.
You need to change all paths according to your locations and the dataset structure described on the project page needs to be used.


In [40]:
## create database .pkl files
LYFT_TRAINVAL_DATASET_ROOT = '/home/itiv/Desktop/lyft-kaggle-dataset/train'
!python Lyft-Detector/second.pytorch/second/create_data.py nuscenes_data_prep --root_path=$LYFT_TRAINVAL_DATASET_ROOT  --version="v1.0-trainval" --dataset_name="NuScenesDataset" --max_sweeps=10


/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environme

In [41]:
## create Ground-Truth .json files 


import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from tqdm import tqdm_notebook as tqdm
import pickle
from pathlib import Path
from nuscenes.nuscenes import NuScenes
from scipy.spatial.transform import Rotation as R 
from math import cos, sin, pi
from lyft_dataset_sdk.lyftdataset import *
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision


LYFT_DATASET_ROOT = LYFT_TRAINVAL_DATASET_ROOT


phase = 'train'
version = 'v1.0-trainval' if phase=='train' else 'v1.0-test'
nusc = NuScenes(dataroot=f'{LYFT_DATASET_ROOT}/', version=version, verbose=0) 
lyft=nusc


#info_path = f'{LYFT_DATASET_ROOT}/infos_val.pkl'
info_path = f'{LYFT_DATASET_ROOT}/infos_train.pkl'
# info_path = '../../../data/nuscenes/v1.0-trainval/infos_train.pkl'
#info_path = '../../../data/nuscenes/v1.0-trainval/infos_val.pkl'
with open(info_path, 'rb') as f:
    info = pickle.load(f)['infos']
    
# only for nuscenes
NameMapping = {
    'movable_object.barrier': 'barrier',
    'vehicle.bicycle': 'bicycle',
    'vehicle.bus.bendy': 'bus',
    'vehicle.bus.rigid': 'bus',
    'vehicle.car': 'car',
    'vehicle.construction': 'construction_vehicle',
    'vehicle.motorcycle': 'motorcycle',
    'human.pedestrian.adult': 'pedestrian',
    'human.pedestrian.child': 'pedestrian',
    'human.pedestrian.construction_worker': 'pedestrian',
    'human.pedestrian.police_officer': 'pedestrian',
    'movable_object.trafficcone': 'traffic_cone',
    'vehicle.trailer': 'trailer',
    'vehicle.truck': 'truck'
}

gt_data = [] 
for inf in tqdm(info):
    sample_token = inf['token']
    sample = lyft.get('sample', sample_token)
    for ann_token in sample['anns']:
        ann_record = lyft.get('sample_annotation', ann_token)
        try:
            data = {
                'sample_token': sample_token,
                'translation': ann_record['translation'],
                'size': ann_record['size'],
                'rotation': ann_record['rotation'],
                'name': ann_record['category_name']
                #'name': NameMapping[ann_record['category_name']]
            }
           # print(ann_record['category_name'])
            gt_data.append(data)
        except Exception as e:
            pass # for nuscenes, not using some categories

        
gt_data_path = f'{LYFT_DATASET_ROOT}/{version}/gt_data_val.json'
with open(gt_data_path, 'w') as f:
    json.dump(gt_data, f, indent=2)
    
    
print(f'Ground-Truth objects saved to: {gt_data_path}')

100%|██████████| 18522/18522 [00:01<00:00, 16781.76it/s]


Ground-Truth objects saved to: /home/itiv/Desktop/lyft-kaggle-dataset/train/v1.0-trainval/gt_data_val.json


In [70]:
##start training
##takes very long better run this in terminal...

config_path = 'Lyft-Detector/second.pytorch/second/configs/nuscenes/all.pp.lowa_large_range_v3.config'
model_path = 'Lyft-Detector/second.pytorch/second/model/model_large_range_v3'

!python ./Lyft-Detector/second.pytorch/second/pytorch/train.py train --config_path=$config_path --model_dir=$model_path

/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environme

## Generate Detections for Scene

In [18]:
##Choose a scene..first load dataset
from lyft_dataset_sdk.lyftdataset import LyftDataset

data_path = LYFT_TRAINVAL_DATASET_ROOT
json_path = data_path+'/v1.0-trainval'
# Load the dataset
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps, v1.0.1-train
level5data = LyftDataset(data_path=data_path, json_path=json_path, verbose=True)

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 6.8 seconds.
Reverse indexing ...
Done reverse indexing in 2.2 seconds.


In [20]:
##Choose a scene..
#level5data.list_scenes()


scene = level5data.scene[10]
print(scene)

{'description': '', 'first_sample_token': '779fb76f1b70ff92cf586bf2ed5f8ac0e9cf5c28fde2c33760cd3219840e529d', 'name': 'host-a011-lidar0-1236119823299280856-1236119848199397346', 'log_token': 'af749b4da1ab470bd4af7bab1a1e9ea358349839844de30291ec3c1c15f94f48', 'last_sample_token': 'a3ad6fd0f3063171545e575a93f0c2e2cabd1bf9bf16e00edbcbbc27540f62c9', 'nbr_samples': 126, 'token': 'af749b4da1ab470bd4af7bab1a1e9ea358349839844de30291ec3c1c15f94f48'}


In [54]:
## create database .pkl files
scene_token = scene['token']

#TODO - create dummy ann files
#LYFT_TRAINVAL_DATASET_ROOT = '/home/itiv/Desktop/lyft-kaggle-dataset/test'
#!python Lyft-Detector/second.pytorch/second/create_data.py nuscenes_data_prep --root_path=$LYFT_TRAINVAL_DATASET_ROOT  --version="v1.0-test" --dataset_name="NuScenesDataset" --scene_token=$scene_token --max_sweeps=10
version = "v1.0-trainval"

LYFT_TRAINVAL_DATASET_ROOT = '/home/itiv/Desktop/lyft-kaggle-dataset/train'
!python Lyft-Detector/second.pytorch/second/create_data.py nuscenes_data_prep --root_path=$LYFT_TRAINVAL_DATASET_ROOT  --version=$version --dataset_name="NuScenesDataset" --scene_token=$scene_token --max_sweeps=10




[autoreload of second.data.nuscenes_dataset failed: Traceback (most recent call last):
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/itiv/Desktop/repo/scenarios_in_CarMaker/BA_Daniel/Lyft-Detector/second.pytorch/s

/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environme

In [60]:
##Perform detection


config_path = "./Lyft-Detector/second.pytorch/second/configs/nuscenes/all.pp.lowa_large_range_v2.config"
ckpt_path = './Lyft-Detector/second.pytorch/second/model/model_large_range_v2/voxelnet-33445.tckpt'
info_path = LYFT_TRAINVAL_DATASET_ROOT+'/infos_'+scene_token+'.pkl'
root_path = LYFT_TRAINVAL_DATASET_ROOT
result_path = LYFT_TRAINVAL_DATASET_ROOT
print('Detecting...this might take several minutes...')
!python ./Lyft-Detector/second.pytorch/second/detect.py detect --scene_token=$scene_token --config_path=$config_path --ckpt_path=$ckpt_path --info_path=$info_path --root_path=$root_path --result_path=$result_path


print('Finished!!!')

Detecting...this might take several minutes...
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/num

In [69]:
#Perform Tracking

root_path = LYFT_TRAINVAL_DATASET_ROOT
result_path = LYFT_TRAINVAL_DATASET_ROOT
version = "v1.0-trainval"
detection_file = result_path+'/tracks_'+scene_token+'.json'
!python ./Lyft-Tracker/mahalanobis_3d_multi_object_tracking/main.py detect --save_root=$root_path --version=$version --detection_file=$detection_file --data_root=$root_path

track nuscenes
Loading NuScenes tables for version v1.0-trainval...
9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 6.6 seconds.
Reverse indexing ...
Done reverse indexing in 2.2 seconds.
Loaded results from /home/itiv/Desktop/lyft-kaggle-dataset/train/tracks_af749b4da1ab470bd4af7bab1a1e9ea358349839844de30291ec3c1c15f94f48.json. Found detections for 126 samples.
  0%|                                                   | 0/126 [00:00<?, ?it/s]./Lyft-Tracker/mahalanobis_3d_multi_object_tracking/main.py:116: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function roty failed at nopython mode lowering due to: 

File "Lyft-Tracker/mahalanobis_3d_multi_object_tracking/main.py", line 123:
def roty(t):
    <source elided>
                     [0,  1,  0],
                     [-s, 0,  c]])
  

In [ ]:
##TODO to carmaker files...